In [2]:
import praw
import re
import random
import bs4, markdown
from tqdm import tqdm
import copy

In [4]:
credential_file = "credentials.key"
target_mbti = "INTP"
try:
    with open(credential_file, 'r') as f:
        creds = f.read().split('\n')
    personal = creds[0]
    secret = creds[1]
    username = creds[2]
    password = creds[3]
except IOError as e:
    print("You didn't create a credential file! Please see sample_credentials.key")
    print("Then go to http://www.storybench.org/how-to-scrape-reddit-with-python/")
    print("And register a new app named fastai_reddit in your reddit account.")
    print("And insert the values into sample_credentials.key and save it as {}.".format(credential_file))
    raise(e)

In [14]:
all_mbti = set(['INTP','INTJ','INFJ','INFP',\
               'ENTP','ENTJ','ENFJ','ENFP',\
               'ESTP','ESTJ','ESFJ','ESFP',
               'ISTP','ISTJ','ISFJ','ISFP'])
len(all_mbti)

16

In [16]:
bad_mbti = copy.copy(all_mbti)
bad_mbti.remove(target_mbti)
bad_mbti = list(bad_mbti)
bad_mbti

['ESTJ',
 'ISTP',
 'ISTJ',
 'ENTJ',
 'ISFJ',
 'ESTP',
 'ISFP',
 'ENFP',
 'ENFJ',
 'ESFJ',
 'INTJ',
 'ESFP',
 'INFP',
 'INFJ',
 'ENTP']

In [6]:
def noquotes(text):
    """
This function first stated out as a way to remove markdown quotes from raw reddit markdown text but now it's more of a
general purpose text parser, but the name hasn't changed.
    """
    #https://stackoverflow.com/questions/761824/python-how-to-convert-markdown-formatted-text-to-text
    t1 = re.sub(">.+?(\n|$)","",text).replace("\\n","").replace("\\","")
    html = markdown.markdown(t1)
    t2 = ''.join(bs4.BeautifulSoup(html, 'lxml').findAll(text=True))
    
    return t2

In [7]:
reddit = praw.Reddit(client_id=personal, client_secret=secret, user_agent='fastai_reddit', username=username, \
                     password=password)
subreddit = reddit.subreddit(target_mbti)

In [8]:
test = list(subreddit.top(limit=10))[0]
print(test.title)

It just can't be that simple


In [17]:
posts = list(subreddit.top(limit=10))

In [26]:
def query_flair(c):
    result = True
    flair = c.author_flair_text
    if not flair:
        return True
    if len(flair) > 0:
        flairU = flair.upper()
        for mbti in bad_mbti:
            if mbti in flairU:
                result = False
                break
    return result

In [27]:
def get_comment_text(comment):
    comments = []
    if isinstance(comment, praw.models.MoreComments):
        newcomments = comment.comments()
        for n in newcomments:
            comments += get_comment_text(n)
    else:
        if query_flair(comment):
            comments.append(noquotes(comment.body))
    return comments
            

In [28]:
alltext = []
for p in tqdm(posts):
    tmp = noquotes(p.selftext)
    if len(tmp) > 0:
        if query_flair(p):
            alltext.append(tmp)
    for c in p.comments.list():
        alltext += get_comment_text(c)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 16.04it/s]


In [24]:
alltext

['What if his parents named him "4" and his sister "2" and he recently changed his name to "44"? It would make his question a non sequitur and indeterminable without more precise information.',
 'Great answer to what was an incredibly stupid question.',
 'I like how it starts with "this is a tough one" and in the end you realise  he\'s half serious actually.',
 'Smart. I like this guy.',
 "4 divided by 2 is 2, so you just divide 44 by 2 so obviously she's 22. Simple math no?",
 "Every time someone asks how many years older or younger someone is than me, I just tell them I actually can't figure it out, unless I really think about it for a minute. I also have to know their birthday. And it just gets so confusing, and I just don't bother",
 'I see a lot of people not understanding that this guy is joking. Nobody actually puts this much thought into a question like this guys, come on.',
 'Well if he was 4 and his sister was 2, then they are both 44.',
 'Studied at Harvard but wastes his th

In [29]:
len(alltext)

1305